In [1]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob


import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository

C:\Users\User\anaconda3\lib\site-packages\pydantic\_internal\_fields.py:128: UserWarning: Field "model_directory" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


In [2]:
connection = sqlite3.connect(settings.db_name, check_same_thread=False)
repo = SQLRepository(connection=connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

repo type: <class 'data.SQLRepository'>
repo.connection type: <class 'sqlite3.Connection'>


In [3]:
settings.model_directory

'models'

In [4]:
from model import GarchModel

# Instantiate a `GarchModel`
gm_spy = GarchModel(ticker="SPY", repo=repo, use_new_data=False)



In [5]:
dir(gm_spy)

['_GarchModel__clean_prediction',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'dump',
 'fit',
 'load',
 'model_directory',
 'predict_volatility',
 'repo',
 'ticker',
 'use_new_data',
 'wrangle_data']

In [6]:
# Instantiate `GarchModel`, use new data
model_shop = GarchModel(ticker="VTI", repo=repo, use_new_data=True)

# Wrangle data
model_shop.wrangle_data(n_observations=2500)


model_shop.data.head()

date
2014-05-28   -0.110954
2014-05-29    0.535191
2014-05-30    0.020088
2014-06-02    0.120506
2014-06-03   -0.040120
Name: return, dtype: float64

In [7]:
# Instantiate `GarchModel`, use old data
model_shop = GarchModel(ticker="VTI", repo=repo, use_new_data=False)

# Wrangle data
model_shop.wrangle_data(n_observations=2500)

# Fit GARCH(1,1) model to data
model_shop.fit(p=1, q=1)


# Check model parameters
model_shop.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -3227.85
Distribution:                  Normal   AIC:                           6463.70
Method:            Maximum Likelihood   BIC:                           6486.99
                                        No. Observations:                 2500
Date:                Thu, May 02 2024   Df Residuals:                     2499
Time:                        04:36:45   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0780  1.493e-02      5.227  1.722e-07 [4.877e-02,  0.107]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0365  9.144e-03      3.995  6.462e-05 [1.861e-02,5.445e-02]
alpha[1]       0.1835  2.783e-02      6.595  4.265e-11     [  0.129,  0.238]
beta[1]        0.7895  2.694e-02     29.306 8.618e-189     [  0.737,  0.842]
============================================================================

Covariance estimator: robust
"""

`Model Fit`: The model's Log-Likelihood, AIC, and BIC suggest a reasonably good fit to the data, as they are within acceptable ranges. However, the R-squared values are not relevant for GARCH models.

`Parameter Significance`: The coefficients in both the mean and volatility models appear to be statistically significant, as indicated by the low p-values.

In [8]:
# Generate prediction from `model_shop`
prediction = model_shop.predict_volatility(horizon=30)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

#prediction

# Convert prediction dictionary to DataFrame
df_prediction = pd.DataFrame(prediction.items(), columns=['Date', 'Prediction'])

# Convert 'Date' column to datetime format
df_prediction['Date'] = pd.to_datetime(df_prediction['Date'])

# Set 'Date' column as index
df_prediction.set_index('Date', inplace=True)

# Display the DataFrame
df_prediction

,Prediction
Date,
2024-05-02,0.945729
2024-05-03,0.952265
2024-05-06,0.958581
2024-05-07,0.964688
2024-05-08,0.970593
2024-05-09,0.976304
2024-05-10,0.981829
2024-05-13,0.987176
2024-05-14,0.992350


In [9]:
# Instantiate `GarchModel`, use new data
model_shop = GarchModel(ticker="SPY", repo=repo, use_new_data=True)


# Wrangle data
model_shop.wrangle_data(n_observations=2500)


model_shop.data.head()

date
2014-05-28   -0.073099
2014-05-29    0.517295
2014-05-30    0.161148
2014-06-02    0.114179
2014-06-03   -0.051840
Name: return, dtype: float64

In [10]:
# Instantiate `GarchModel`, use old data
model_shop = GarchModel(ticker="SPY", repo=repo, use_new_data=False)

# Wrangle data
model_shop.wrangle_data(n_observations=2500)

# Fit GARCH(1,1) model to data
model_shop.fit(p=1, q=1)

# Check model parameters
model_shop.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -3174.75
Distribution:                  Normal   AIC:                           6357.51
Method:            Maximum Likelihood   BIC:                           6380.81
                                        No. Observations:                 2500
Date:                Thu, May 02 2024   Df Residuals:                     2499
Time:                        04:36:47   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0815  1.449e-02      5.626  1.846e-08 [5.311e-02,  0.110]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0355  8.840e-03      4.016  5.909e-05 [1.818e-02,5.283e-02]
alpha[1]       0.1901  2.806e-02      6.776  1.239e-11     [  0.135,  0.245]
beta[1]        0.7841  2.602e-02     30.138 1.556e-199     [  0.733,  0.835]
============================================================================

Covariance estimator: robust
"""

In [11]:
# Generate prediction from `model_shop`
prediction = model_shop.predict_volatility(horizon=30)

#prediction

# Convert prediction dictionary to DataFrame
df_prediction = pd.DataFrame(prediction.items(), columns=['Date', 'Prediction'])

# Convert 'Date' column to datetime format
df_prediction['Date'] = pd.to_datetime(df_prediction['Date'])

# Set 'Date' column as index
df_prediction.set_index('Date', inplace=True)

# Display the DataFrame
df_prediction

,Prediction
Date,
2024-05-02,0.931344
2024-05-03,0.938375
2024-05-06,0.945173
2024-05-07,0.951750
2024-05-08,0.958113
2024-05-09,0.964273
2024-05-10,0.970235
2024-05-13,0.976009
2024-05-14,0.981602


## COMMUNICATION OF RESULT

`Comparison of Volatility`: The forecasted volatility indicate the expected volatility of VTI and SPY over the specified time period. VTI has higher forecasted volatility compared to SPY on each day, suggesting that VTI is expected to be more volatile than SPY during this time frame.

`Diversification`: Investors seeking to manage risk may consider diversifying their portfolios by holding a combination of assets with different levels of volatility. By diversifying across assets with low, moderate, and high volatility, investors can potentially reduce the overall risk of their portfolio while still seeking returns.

`Long-Term Perspective`: While volatility can be concerning in the short term, investors with a long-term perspective may be less affected by short-term fluctuations. They should focus on the fundamentals of the assets, such as their historical performance, underlying business prospects, and alignment with their investment goals, rather than being swayed by short-term volatility.

`Risk Management Strategies`: Investors concerned about volatility can implement risk management strategies, such as using stop-loss orders, hedging with options or futures contracts, or allocating a smaller portion of their portfolio to volatile assets.

`Regular Monitoring`: It's essential for investors to regularly monitor the volatility and risk profile of their investments, as market conditions and asset dynamics can change over time. By staying informed and adapting their investment strategies accordingly, investors can better manage risk and optimize their investment returns.



##### The fact that the forecasted variance of the SPDR S&P 500 ETF Trust (SPY) is lower than that of Vanguard Total Stock Market ETF (VTI) may contribute to its status as the market standard .

   The lower forecasted volatility of SPY suggests that it is expected to exhibit lower volatility compared to VTI. Lower volatility can be appealing to investors who prioritize stability and seek to minimize the potential for large price fluctuations in their investments. As a result, SPY's lower volatility may attract more risk-averse investors, contributing to its widespread acceptance as a market standard.
    
   SPY's lower forecasted volatilty may enhance its perceived stability relative to VTI. Investors often view stability positively, especially during periods of market uncertainty or heightened volatility. SPY's reputation for stability can reinforce its status as a preferred choice for benchmarking investment performance and assessing market trends.
   
   As the market standard for tracking the performance of the S&P 500 Index, SPY is widely used as a benchmark for evaluating the performance of other investments, such as mutual funds, other ETFs, and individual stocks. The lower forecasted volatility of SPY relative to VTI provides a reference point for investors to compare the risk and volatility characteristics of different investment options, further solidifying SPY's position as the market standard.
   
   SPY's historical performance and track record of lower volatility compared to broader market indices may contribute to its perception as a reliable and consistent investment option. Investors often value consistency in investment products, and SPY's ability to maintain lower volatility over time may reinforce its position as the market standard.
   
   SPY's status as the market standard is also supported by its high liquidity and trading activity, which are essential characteristics for an ETF to serve as an effective benchmark. The lower forecasted volatility of SPY may attract more trading activity and investor interest, further solidifying its position as the preferred choice for tracking the performance of the broader market.
   
   In summary, the lower forecasted volatilty of SPY relative to VTI contributes to its status as the market standard by enhancing its perceived stability, attractiveness to risk-averse investors, suitability for benchmarking and comparison, historical consistency, and liquidity. These factors collectively reinforce SPY's reputation as the primary reference point for evaluating the performance of the U.S. stock market.
